In [2]:
import numpy as np
import cupy as cp
import polars as pl

In [3]:
# Last 3 OHLCV candles normalized between 0 and 1 by Lmin and Hmax
# y = return between close(t) and close(t+1)
# Stock identified by "Tick"
candles = pl.read_csv("../trading_data/FTM/candles.csv")

In [5]:
candles.head()

Tick,Date,O0,O1,O2,H0,H1,H2,L0,L1,L2,C0,C1,C2,V0,V1,V2,y
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A""","""2010-01-06""",0.097056,0.517632,0.7224,0.274992,0.533808,1.0,0.0,0.0,0.423326,0.097056,0.226464,0.618247,0.774922,1.0,0.911498,-0.0013
"""A""","""2010-01-07""",0.384615,0.480769,0.980769,0.442308,0.692308,1.0,0.0,0.365385,0.365385,0.423077,0.480769,0.634615,0.739396,0.774922,1.0,-0.000454
"""A""","""2010-01-08""",0.395349,0.627907,0.744186,0.744186,0.697674,1.0,0.0,0.162791,0.604651,0.651163,0.674419,0.744186,1.0,0.8289,0.868747,0.000908
"""A""","""2010-01-11""",0.7391,0.369565,0.586957,1.0,0.695652,0.652174,0.413043,0.0,0.152174,0.652174,0.608696,0.630435,1.0,0.780893,0.647298,-0.0122
"""A""","""2010-01-12""",0.4067,0.7966,0.508475,0.525424,1.0,0.762712,0.0,0.542373,0.220339,0.271186,0.728814,0.694915,0.600445,1.0,0.780893,0.008268


In [8]:
# FTM data
# Stock identified by "Tick"
ftm = pl.read_csv("../trading_data/FTM/FTM.csv")

In [9]:
ftm.head()

Tick,Date,MOM1,MOM2,MOM3,RET_SINCE_YEST_CLOSE,RET_SINCE_BOD,RET_FROM_DAYS_LOW,RET_FROM_DAYS_HIGH,CURRENT_PRICE,fwdret
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""A""","""2010-05-12""",0.0153,0.0009,0.067159,0.018014,0.015222,0.016115,-0.0088,34.68,0.003171
"""A""","""2010-05-13""",0.0096,0.000127,0.0647,0.006689,0.005881,0.018443,-0.002844,35.065,-0.008697
"""A""","""2010-05-14""",-0.011439,-0.024639,0.035788,-0.033893,-0.022972,0.004484,-0.031979,33.6,0.008332
"""A""","""2010-05-17""",-0.001176,-0.014132,0.044872,0.002477,-0.002343,0.002059,-0.012754,34.06,0.003523
"""A""","""2010-05-18""",-0.043089,-0.070901,-0.019653,-0.043742,-0.038235,0.008948,-0.039648,32.7,-0.019569


In [10]:
# Raw daily data
# Stock identified by "Tick"
xpsd = pl.read_csv("../trading_data/FTM/XPSD.csv")

In [11]:
xpsd.head()

D,O,H,L,C,V,Tick
str,f64,f64,f64,f64,i64,str
"""2000-01-03""",56.3131,56.4472,48.179,51.4863,3343300,"""A"""
"""2000-01-04""",48.7153,49.2516,46.3019,46.8829,3408200,"""A"""
"""2000-01-05""",47.3745,47.3745,43.1287,44.6035,4119199,"""A"""
"""2000-01-06""",44.0672,44.3354,41.5644,42.1901,1812800,"""A"""
"""2000-01-07""",42.2348,47.151,42.1901,46.4806,2016900,"""A"""
